# Exploring Python Performance with Computational Fluid Dynamics

### Introduction

This exercise takes an example from one of the most common applications of HPC resources: Fluid Dynamics. We will look at how a simple fluid dynamics problem can be run using Python with Numpy, how Fortran and/or C code can be
called from within Python, and how Numba can be used gain speedup from just-in-time compilation. The exercise will compare the performance of the different approaches.


### Fluid Dynamics: a brief overview

Fluid Dynamics is the study of the mechanics of fluid flow, liquids and gases in motion. This can encompass aerodynamics and hydrodynamics. It has wide ranging applications from vessel and structure design to weather and traffic modelling. Simulating and solving fluid dynamic problems often requires large computational resources.

Fluid dynamics is an example of continuous system that can be described by Partial Differential Equations. For a computer to simulate these systems, the equations must be discretised onto a grid. If this grid is regular, then a finite difference approach can be used. Using this method means that the value at any point in the grid is updated using some combination of the neighbouring points.

_Discretisation_ is the process of approximating a continuous (i.e. infinite-dimensional) problem by a finite-dimensional problem suitable for a computer. This is often accomplished by putting the calculations into a grid or similar construct.

### The Problem

In this exercise the finite difference approach is used to determine the flow pattern of a fluid in a cavity. For simplicity, the liquid is assumed to have zero viscosity, which implies that there can be no vortices (i.e. no whirlpools) in the flow. The cavity is a square box with an inlet on one side and an outlet on another as shown below.

![cavity image](./images/cavity.png)


#### Mathematical background (optional)

In two dimensions it is easiest to work with the stream function
$\psi$ (see below for how this relates to the fluid velocity). For zero viscosity, $\psi$ satisfies the following equation:

$$
\nabla^2 \psi = \frac{\partial^2 \psi}{\partial x^2}
$$

The finite difference version of this equation is:

$$
\psi_{i-1,j} + \psi_{i+1,j} + \psi_{i,j-1} + \psi_{i,j+1}
-4 \psi_{i,j} = 0.
$$

With the boundary values fixed, the stream function can be calculated for each point in the grid by averaging the value
at that point with its four nearest neighbours. The process continues until the algorithm converges on a solution that
stays unchanged by the averaging process. This simple approach
to solving a PDE is called the Jacobi algorithm.

In order to obtain the flow pattern of the fluid in the cavity
we want to compute the velocity field $\mathbf{u}(x,y)$. The $x$ and $y$ components of the velocity are related to the stream function by

$$
u_x =  \frac{\partial \psi}{\partial y} = \frac{1}{2}(\psi_{i,j+1} - \psi_{i,j-1}),
\quad
u_y = -\frac{\partial \psi}{\partial x} = \frac{1}{2}(\psi_{i+1,j}-\psi_{i-1,j}).
$$

This means that the velocity of the fluid at each grid point
can also be calculated from the surrounding grid points. The magnitude of the velocity $\mathbf{u}$ is
given by $u = (u_x^2 + u_y^2)^{1/2}$.

### An algorithm

The outline of the algorithm for calculating the velocities is
as follows:

```
Set the boundary values for stream function
while (convergence is FALSE):
     for each interior grid point:
         update the stream function
     
     compute convergence criteria

for each interior grid point:
    compute x component of velocity
    compute y component of velocity
```

For simplicity, here we simply run the calculation for a fixed number of iterations; a real simulation would continue until
some chosen accuracy was achieved.

### Using Python

You are given a basic code in `./code/python_lists` that uses Python lists to run the simulation. 
There are a number of different files:

```
cfd.py           # python driver script
jacobi.py        # Jacobi algorthm code
plot_flow.py     # separate script to plot flow
util.py          # utility functions
```
Look at the structure of the `cfd.py` code. In particular, note:

* How the external "jacobi" function is included
* How the lists are declared and initialised to zero
* How the timing works

### First Run and Verification

First, verify that your copy of the code is producing the correct results.

Navigate to the `python_lists` subdirectory and run the main program with:
```bash
prompt:~/python_lists> ./cfd.py 1 1000
```

This runs the CFD simulation with a scale factor of 1 and 1000 Jacobi iteration steps. As the program is running you should see output that looks something like:
```

2D CFD Simulation
=================
Scale factor = 1
Iterations   = 1000

Grid size = 32 x 32

Starting main Jacobi loop ...

completed iteration 1000

... finished

Calculation took 1.11856s
```

The program will produce two text output files called `velocity.dat` and `colourmap.dat` with the computed velocities
at each grid point, and data providing a representation of the velocity magnitude, respectively. A simple verification is to use diff to compare your output with one of the verification datasets. For example:

```
prompt:~> diff velocity.dat ../verify/cfd_velocity_1_1000.dat
```

`diff` will only produce any output if it finds any differences between the two files. If you see any differences at this point, please ask a tutor.

### Initial benchmarking

Now produce some baseline figures with which to compare your future versions of the code. You should pick a set of representative problem sizes (defined by scale and number of iterations) that run in a sensible time on your machine but do not complete instantaneously. (A good place to start is with scale factor 2 and 5000 iterations. You will also need some smaller and larger examples.)

Record the benchmarking calculation times for future reference.

The directory includes a utility called `plot_flow.py` that produces a graphical representation of the final state of the simulation. You can use this to produce a PNG image as follows:

```bash
prompt:~> ./plot_flow.py velocity.dat colourmap.dat flow.png
```
The resulting image should look something like this:
![flow image](./images/reference.png)

If the fluid is flowing around the edge of the domain, rather than through the middle of the cavity, then this is an indication that the Jacobi algorithm has not yet converged. Convergence requires more iterations on larger problem sizes.

### Using Numpy arrays

We will now refactor the CFD code to use Numpy arrays rather than Python lists. This has a number of advantages:

* Numpy is closely integrated with matplotlib and using Numpy arrays will allow us to produce the visualisation directly from our simulation rather than using a separate utility.

* Numpy arrays should allow us to access better performance using more concise code.

* Numpy arrays are directly compatible with native code produced by Fortran and C compilers. This will allow us to re-code the key part of our algorithm and achieve better performance while still having the benefits of coding in Python.

Copy `cfd.py` and `jacobi.py` into the directory `python_numpy`, and replace the `psi` and `tmp` lists in the code with Numpy arrays. You will need the statement:

```python
import numpy as np
```

at the top of all your source files to ensure you can access the Numpy functionality. The arrays will need to be implemented in the main function and all of the other functions where they are used.

Declaring and zeroing Numpy arrays can be done in a single statement such as:

```python
psi = np.zeros((m+2, n+2))
```

You may find it useful to refer to the following:

https://numpy.org/doc/stable/user/quickstart.html

https://numpy.org/doc/stable/user/basics.indexing.html

https://numpy.org/doc/stable/user/basics.broadcasting.html

Once you think you have the correct code, run your new program and compare the output with that from the original code. If you are convinced that the output is the same then move on and benchmark your new code.

What do you find? Has using Numpy arrays increased the performance of the code? Can you think of an explanation of why the performance has altered in the way that it has?

Can you change the implementation to produce a better performing version of the CFD code?

Hint 1: which method of accessing 2D array elements is faster: `a[i][j]` or `a[i,j]`?

Hint 2: you should use array index syntax (or _slicing_) if you have not already done so to specify blocks of arrays to operate on.

### Incorporating matplotlib

The packages `matplotlib` and `numpy` have a very close relationship: `matplotlib` can use the type and structure of numpy `arrays` to simplify the plotting of data. You have been provided with a function called `plot_data` in the file `matplot_flow.py`, which will plot an image of the final state of the flow and which you can import into your code to use. Define a function in your main `cfd.py` file called, e.g., `plot_flow()`. This function takes two arguments: the first is `psi` the `numpy` array containing the stream function values and the second is the name of the file in which to save the image.


### Using `scipy`

You should have found that the Numpy implementation that simply iterates using explicit `range()` loops does not much improve performance compared with bare Python lists. The implementation using slicing will give better performance (why?). 

The `scipy` function `ndimage.convolve()` provides one way to improve performance, whilst maintaining readability.

Write a version of the `jacobi.py` routine that uses the `convolve()` function and assess its performance against your previous versions.

Note you will need to define the mask to use for the convolution. This is essentially a stencil that you place over the current element that describes how to combine the surrounding elements to produce the required operation previously implemented using for loops or indexing. To write this version of the function you will need to design your stencil and express it in the code as a 2D numpy array.

Consult the online reference documentation for `convolve()` to work out how to do this:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html

In the terminology of the documentation, the input array is our stream function $\psi$ and the weights correspond to the stencil. 

### Using just-in-time compilation with Numba

One other way to improve performance is to use the `numba` package, which uses LLVM to perform just-in-time compilation of selected functions of your code that you need to indicate the function decorator `@jit`.

One can try, e.g.,
```python
from numba import jit

@jit(nopython = True)
def jacobi(niter, psi):
    ...
```
What happens? Can you refactor the code to get the compilation to work? How does the time compare with, e.g., the sliced version of the algorithm?


### Calling external code from Python

We are going to continue using our Numpy implementation of the CFD code to illustrate calling C or Fortran code from Python. Calling any external code (written in any language) from Python requires
* data passing from python has a known size and layout
* the external routines have an interface which can be imported in python

We must also have an appropriate compiler. Most platforms will allow the installation of and least GNU C. 

Using Numpy arrays will help to ensure that data can be passed correctly to and from Python. The Python packages `ctypes` and `f2py` provide ways to do this and you are free to choose whichever you prefer by following the appropriate instructions linked below. 

#### Using C and ctypes

See the instructions [README-c-ctypes.ipynb](./README-c-ctypes.ipynb)

#### Using Fortran and f2py

See the instructions [README-f-f2py.ipynb](./README-f-f2py.ipynb)
